Transition probability matrix after n time steps is given by:
$ p_n = qP^n $ where q is initial probability distribution. 

We will calculate the transition probability matrix for the given scenario:
<br>
<img src="robot.png" style="width: 300px;"/>

But instead of a 4x4 environment, we will use 3x3 and robot will start at (1, 1)

## First, we will calculate the $p_n$ using transition probabilities

In [1]:
import numpy as np
m = 3
m2 = m**2

q = np.zeros(m2)
q[4] = 1

In [2]:
def get_P(m, p_up, p_down, p_left, p_right):
    m2 = m**2
    P = np.zeros((m2, m2))
    
    # this statement calculates indices for each location
    index_map = {i + 1: (i // m, i % m) for i in range(m2)}
    
    for i in range(m2):
        for j in range(m2):
            r1, c1 = index_map[i + 1]
            r2, c2 = index_map[j + 1]
            # we are transitioning from (r1, c1) to (r2, c2)
            
            r_diff = r1 - r2
            c_diff = c1 - c2
            
            if r_diff == 0:
                if c_diff == 1:
                    P[i, j] = p_left
                elif c_diff == -1:
                    P[i, j] = p_right
                elif c_diff == 0:
                    if r1 == 0:
                        P[i, j] += p_down
                    elif r1 == m - 1:
                        P[i, j] += p_up
                    
                    if c1 == 0:
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
                        
            elif r_diff == 1:
                if c_diff == 0:
                    P[i, j] = p_down
            elif r_diff == -1:
                if c_diff == 0:
                    P[i, j] = p_up
    
    return P

In [3]:
P = get_P(3, 0.2, 0.3, 0.25, 0.25)

In [4]:
n = 1
Pn = np.linalg.matrix_power(P, n)
result = q @ Pn

# flipping just for good visualization
np.flip(result.reshape(m, m), axis=0)

array([[0.  , 0.2 , 0.  ],
       [0.25, 0.  , 0.25],
       [0.  , 0.3 , 0.  ]])

In [5]:
n = 10
Pn = np.linalg.matrix_power(P, n)
result = q @ Pn

# flipping just for good visualization
np.flip(result.reshape(m, m), axis=0)

array([[0.07180907, 0.07154579, 0.07180907],
       [0.10541537, 0.10600186, 0.10541537],
       [0.15610857, 0.15578632, 0.15610857]])

## If Markov chain is ergodic, we can simulate and estimate steady-state distribution

In [6]:
state = 4
n = 10 ** 6
visited = [4]

In [7]:
for _ in range(n):
    state = np.random.choice(m2, p=P[state, :])
    visited.append(state)

In [8]:
np.unique(visited, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([157571, 158165, 159053, 104374, 104797, 105638,  70135,  69724,
         70544]))

## Now lets calculate the average reward for each state with simulation, which is state values in this case

Reward is defined by +1 for each transaction. Hitting the wall ends the episode.

We first modify the transition probability matrix:

In [9]:
P = np.zeros((m2 + 1, m2 + 1))
P[:m2, :m2] = get_P(3, 0.2, 0.3, 0.25, 0.25)

# Initially, we made it bouncing from walls but now we are changing the bouncing probabilites to hitting probabilities
for i in range(m2):
    P[i, m2] = P[i, i]
    P[i, i] = 0

P[m2, m2] = 1

Next we simulate the environment by select next states based on transition probability matrix and calculate average reward for each state:

In [18]:
n = 10 ** 5
avg_rewards = np.zeros(m2)

for s in range(m2):
    for i in range(n):
        crashed = False
        episode_reward = 0
        s_next = s
        
        while not crashed:
            s_next = np.random.choice(m2 + 1, p=P[s_next, :])
            if s_next < m2:
                episode_reward += 1
            else:
                crashed = True
            
        avg_rewards[s] += episode_reward

avg_rewards /= n

In [19]:
np.flip(avg_rewards.reshape(m, m), axis=0)

array([[1.99882, 2.83507, 1.99706],
       [2.43297, 3.41346, 2.43251],
       [1.46378, 2.1139 , 1.4701 ]])

## We can also analtically calculate the state values using Bellman equations

We know that $v(s) = \sum_{s',r}p(s', r|s)[r+\gamma v(s')]$

When we incorporate it with transition probabilities: $v = PR + \gamma Pv$

And we can solve this equation for v as:
$$
(I - \gamma P)v = PR
$$
$$
v=(I - \gamma P)^{-1}PR
$$

In [21]:
R = np.ones(m2 + 1)
R[-1] = 0

gamma = 0.9999

In [22]:
inv = np.linalg.inv(np.eye(m2 + 1) - gamma * P)
v = inv @ (P @ R)
v

array([1.46796404, 2.11794995, 1.46796404, 2.4428918 , 3.42054869,
       2.4428918 , 1.98767361, 2.81979941, 1.98767361, 0.        ])

Note that these values are real state values. We can see that the result we found earlier is almost same with this